In [1]:
val k = 11
val iterations = 25
val stopWords = Array("the", "a", "and", "with", "holding", "left", "right","two", "extending", "below", "above", "standing", "him", "over", "for", "from", "within", "seated", "before", "behind", "front", 
                      "draped", "cuirassed", "head", "shoulder",
                      "each", "set", "between", "inscribed", "wearing", "raising", "facing", "which", "except", "surronding", "beneath", "who", "other",
                      "hand")
val vocabSize = 15000
val minimumTokenLength = 3

// Cosmetic setting for table display:
val maxWidth = 1000

import scala.io.Source
import scala.io.Source
val url = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/ocre-cite-ids.cex"


val lines = Source.fromURL(url).getLines.toVector
val rawText = lines.map(ln => { val cols = ln.split("#");  cols(4) + " " + cols(10) })

val cleanText = rawText.map(_.toLowerCase).map(_.replaceAll("[^a-z ]", ""))

import $ivy.`org.apache.spark::spark-sql:2.4.5` // Or use any other 2.x version here
import org.apache.spark.sql._
import $ivy.`org.apache.spark::spark-mllib:2.4.5`
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
val spark = {
  NotebookSparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

import org.apache.spark.ml.clustering.LDA
import org.apache.spark.ml.feature.RegexTokenizer
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.CountVectorizer
import org.apache.spark.mllib.linalg.Vector
import scala.collection.mutable.WrappedArray
import org.apache.spark.sql.types.IntegerType
import org.apache.spark.sql.functions._

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

k: Int = 11
iterations: Int = 25
stopWords: Array[String] = Array(
  "the",
  "a",
  "and",
  "with",
  "holding",
  "left",
  "right",
  "two",
  "extending",
  "below",
  "above",
  "standing",
  "him",
  "over",
  "for",
  "from",
  "within",
  "seated",
  "before",
  "behind",
  "front",
  "draped",
  "cuirassed",
  "head",
  "shoulder",
  "each",
  "set",
  "between",
  "inscribed",
  "wearing",
  "raising",
  "facing",
  "which",
  "except",
  "surronding",
  "beneath",
  "who",
  "other",
...
vocabSize: Int = 15000
minimumTokenLength: Int = 3
maxWidth: Int = 1000
import scala.io.Source

import scala.io.Source

url: String = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/ocre-cite-ids.cex"
lines: scala.package.Vector[String] = Vector(
  "ID#Label#Denomination#Metal#Authority#Mint#Region#ObvType#ObvLegend#ObvPortraitId#RevType#RevLegend#RevPortraitId#StartDate#EndDate",
  "3.com.43#RIC III Commodus 43#denarius#ar#commodus#rome#italy#Head of Commodus, laureate, rig

In [2]:
// Create RDD:
val txtRdd = spark.sparkContext.parallelize(cleanText).zipWithIndex

import spark.sqlContext.implicits._

val corpus_df = txtRdd.toDF("corpus", "id")

txtRdd: org.apache.spark.rdd.RDD[(String, Long)] = ZippedWithIndexRDD[1] at zipWithIndex at cmd1.sc:1
import spark.sqlContext.implicits._


corpus_df: DataFrame = [corpus: string, id: bigint]

In [3]:
// based on a snippet by Ivan Zaitsev
// https://github.com/almond-sh/almond/issues/180#issuecomment-364711999
implicit class RichDF(val df: DataFrame) {
  def showHTML(limit:Int = 20, truncate: Int = 20) = {
    import xml.Utility.escape
    val data = df.take(limit)
    val header = df.schema.fieldNames.toSeq
    val rows: Seq[Seq[String]] = data.map { row =>
      row.toSeq.map { cell =>
        val str = cell match {
          case null => "null"
          case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
          case array: Array[_] => array.mkString("[", ", ", "]")
          case seq: Seq[_] => seq.mkString("[", ", ", "]")
          case _ => cell.toString
        }
        if (truncate > 0 && str.length > truncate) {
          // do not show ellipses for strings shorter than 4 characters.
          if (truncate < 4) str.substring(0, truncate)
          else str.substring(0, truncate - 3) + "..."
        } else {
          str
        }
      }: Seq[String]
    }

    publish.html(s"""
      <table class="table">
        <tr>
        ${header.map(h => s"<th>${escape(h)}</th>").mkString}
        </tr>
        ${rows.map { row =>
          s"<tr>${row.map { c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
        }.mkString
        }
      </table>""")
  }
}

defined class RichDF

In [4]:
val tokenizer = new RegexTokenizer().setPattern("[\\W_]+").setMinTokenLength(minimumTokenLength).setInputCol("corpus").setOutputCol("tokens")
val tokenized_df = tokenizer.transform(corpus_df)

val remover = new StopWordsRemover().setStopWords(stopWords).setInputCol("tokens").setOutputCol("filtered")
val filtered_df = remover.transform(tokenized_df)

val vectorizer = new CountVectorizer().setInputCol("filtered").setOutputCol("features").setVocabSize(vocabSize).setMinDF(5).fit(filtered_df)
val countVectors = vectorizer.transform(filtered_df).select("id", "features")

val lda = new LDA().setK(k).setMaxIter(iterations)
val model = lda.fit(countVectors)

flatMap at CountVectorizer.scala:205

1 / 1

count at CountVectorizer.scala:230

1 / 1

top at CountVectorizer.scala:233

...

isEmpty at LDAOptimizer.scala:450

1 / 1

treeAggregate at LDAOptimizer.scala:504

1 / 1

tokenizer: RegexTokenizer = regexTok_dd2e9bc84a38
tokenized_df: DataFrame = [corpus: string, id: bigint ... 1 more field]
remover: StopWordsRemover = stopWords_83db6022a2f7
filtered_df: DataFrame = [corpus: string, id: bigint ... 2 more fields]
vectorizer: org.apache.spark.ml.feature.CountVectorizerModel = cntVec_449c4a4078eb
countVectors: DataFrame = [id: bigint, features: vector]
lda: LDA = lda_477810e488b5
model: org.apache.spark.ml.clustering.LDAModel = lda_477810e488b5

In [5]:
val topics = model.describeTopics(20)
topics.showHTML(truncate=1000)

topic,termIndices,termWeights
0,"[3, 9, 2, 16, 21, 17, 23, 37, 10, 1, 44, 50, 18, 65, 56, 53, 72, 22, 54, 13]","[0.0570027569523144, 0.04926904087489605, 0.03911958972787758, 0.029249465071014613, 0.028174847245859353, 0.026326216152164766, 0.025538307059085538, 0.021262117171857952, 0.017740085529063785, 0.01703733665857046, 0.017017007703640088, 0.015170736951189798, 0.013887522140126623, 0.013270067405996698, 0.01217458401674285, 0.011790767678030112, 0.01106034016293905, 0.01080241235698539, 0.01048187568097295, 0.009936991744028517]"
1,"[112, 448, 450, 664, 666, 559, 801, 695, 923, 469, 865, 368, 181, 680, 811, 449, 860, 859, 777, 727]","[0.07438273957156721, 0.018423537954959573, 0.015220464355684403, 0.0111671614210492, 0.009262540695132845, 0.00827689539992169, 0.007552555453162492, 0.007431307570933221, 0.007197034292504661, 0.006927930920432636, 0.006856398455083203, 0.0066722327643573024, 0.006445188866688572, 0.006318390799468198, 0.0062865898240733655, 0.005997826415343443, 0.00586053853316456, 0.005724334896064792, 0.005664848371410077, 0.00558734265975644]"
2,"[85, 14, 32, 1, 142, 31, 15, 202, 109, 139, 246, 247, 271, 154, 194, 282, 319, 318, 219, 375]","[0.049029496446781194, 0.036386114682854916, 0.03256285194845308, 0.032408812823256354, 0.030497681098475775, 0.02485019589417943, 0.022532477067079847, 0.02070563601565835, 0.018542874170460244, 0.017224997582351975, 0.01708964801944152, 0.016722429966800643, 0.015643746949793428, 0.015554184547806384, 0.013044882128699127, 0.010160540448863763, 0.01007128227719765, 0.009869266242924038, 0.009741870955619262, 0.009708450444701353]"
3,"[1, 0, 12, 6, 8, 36, 11, 20, 52, 62, 83, 3, 22, 40, 78, 27, 63, 13, 39, 163]","[0.05738169550349262, 0.05649919814367649, 0.05253840547466611, 0.05082142208153891, 0.03642126606622154, 0.030048335900069785, 0.025424828723638795, 0.024984171280504886, 0.023408682926464842, 0.02037443954492004, 0.019139390694642842, 0.018757363095869242, 0.01783932735654865, 0.016177396404340974, 0.016038791561847697, 0.015267652547054455, 0.014979324861764327, 0.012232645566130896, 0.00991602215798045, 0.009261319900296276]"
4,"[26, 0, 2, 25, 60, 8, 86, 88, 48, 19, 129, 94, 132, 6, 73, 30, 80, 66, 11, 70]","[0.07109937991988029, 0.06385290035379859, 0.05573777574968814, 0.05432417184679488, 0.03984369639467067, 0.038882132624450824, 0.03323673398758548, 0.03088591959805974, 0.0306908487647795, 0.023895071575307857, 0.02136168606953911, 0.020797572306075072, 0.02032014752432319, 0.018860828585207715, 0.01868272664026337, 0.01630366098908093, 0.01472110250649863, 0.014146710735104127, 0.01262045134205413, 0.011926230869223182]"
5,"[5, 4, 7, 0, 14, 2, 11, 13, 24, 1, 38, 42, 6, 43, 45, 28, 10, 8, 31, 30]","[0.05938984996306914, 0.05727033794472761, 0.03986869605676984, 0.03296967477722263, 0.019258976690932874, 0.018717608836199096, 0.018469164846977946, 0.017212788939039627, 0.016372325053449308, 0.015055195974641764, 0.013365691557876874, 0.012923064442012093, 0.012802028033113526, 0.01256898176851692, 0.01234132095713391, 0.012037409121119629, 0.011753050886446742, 0.010936652542894078, 0.010448681019765586, 0.009848173824575548]"
6,"[1, 51, 69, 35, 3, 131, 87, 58, 41, 15, 2, 96, 210, 19, 79, 49, 201, 111, 197, 239]","[0.060364246966633936, 0.047539406734559, 0.03798798162479387, 0.035714775157328904, 0.033888991040258575, 0.025702160447007353, 0.024008310101981845, 0.022954976310238715, 0.021404696375193148, 0.01981461686654317, 0.018732506904440942, 0.01870839358691975, 0.018417085973427023, 0.01580109384679378, 0.01567393747009918, 0.015395642586528781, 0.014368907490866854, 0.013899343158401132, 0.012442758466654161, 0.012109505755433616]"
7,"[398, 312, 806, 643, 500, 884, 893, 1159, 1103, 39, 1128, 1363, 881, 488, 183, 1222, 1600, 1, 342, 828]","[0.04317958250309407, 0.03600836885619099, 0.013796199587413006, 0.009321254792452136, 0.007518565782922464, 0.007215840939824137, 0.006142416346694171, 0.0053319089327310935, 0.005221882043510673, 0.00

topics: DataFrame = [topic: int, termIndices: array<int> ... 1 more field]

In [6]:
val topicLabels = topics.select("termIndices").map { case Row(r:  WrappedArray[Integer]) => r.map( i => vectorizer.vocabulary(i) ) }
val labelsNumberedLong = topicLabels.rdd.zipWithIndex.toDF("terms", "topicLong")
val labelsIndexed = labelsNumberedLong.withColumn("topic", $"topicLong".cast(IntegerType)).drop("topicLong")

val topicsWithTerms = labelsIndexed.join(topics, labelsIndexed.col("topic") === topics.col("topic")).drop(labelsIndexed.col("topic"))

val weightedLabels = topicsWithTerms.withColumn("termsWithWeight", expr("zip_with(terms, termWeights, (t,w) -> concat(t, ' ', w))"))


topicLabels: Dataset[WrappedArray[String]] = [value: array<string>]
labelsNumberedLong: DataFrame = [terms: array<string>, topicLong: bigint]
labelsIndexed: DataFrame = [terms: array<string>, topic: int]
topicsWithTerms: DataFrame = [terms: array<string>, topic: int ... 2 more fields]
weightedLabels: DataFrame = [terms: array<string>, topic: int ... 3 more fields]

In [7]:
weightedLabels.select("topic", "termsWithWeight").showHTML(truncate=1000)


run at ThreadPoolExecutor.java:1149

1 / 1

take at cmd2.sc:4

1 / 1

topic,termsWithWeight
0,"[cornucopiae 0.0570027569523144, patera 0.04926904087489605, globe 0.03911958972787758, altar 0.029249465071014613, chlamys 0.028174847245859353, nude 0.026326216152164766, genius 0.025538307059085538, modius 0.021262117171857952, constantinei 0.017740085529063785, sceptre 0.01703733665857046, sol 0.017017007703640088, radiate 0.015170736951189798, maximian 0.013887522140126623, out 0.013270067405996698, rudder 0.01217458401674285, togate 0.011790767678030112, fortuna 0.01106034016293905, antoninuspius 0.01080241235698539, cornears 0.01048187568097295, gallienus 0.009936991744028517]"
1,"[augustus 0.07438273957156721, cippus 0.018423537954959573, capricorn 0.015220464355684403, bundle 0.0111671614210492, pegasus 0.009262540695132845, offering 0.00827689539992169, arabia 0.007552555453162492, grain 0.007431307570933221, canes 0.007197034292504661, aeternitas 0.006927930920432636, imp 0.006856398455083203, attached 0.0066722327643573024, carrying 0.006445188866688572, cloaked 0.006318390799468198, stalks 0.0062865898240733655, avg 0.005997826415343443, hippocamp 0.00586053853316456, camel 0.005724334896064792, sun 0.005664848371410077, moon 0.00558734265975644]"
2,"[column 0.049029496446781194, resting 0.036386114682854916, leaning 0.03256285194845308, sceptre 0.032408812823256354, securitas 0.030497681098475775, arm 0.02485019589417943, hadrian 0.022532477067079847, legs 0.02070563601565835, nero 0.018542874170460244, small 0.017224997582351975, crossed 0.01708964801944152, elbow 0.016722429966800643, reclining 0.015643746949793428, figure 0.015554184547806384, bareheaded 0.013044882128699127, lyre 0.010160540448863763, basket 0.01007128227719765, drawing 0.009869266242924038, rock 0.009741870955619262, pudicitia 0.009708450444701353]"
3,"[sceptre 0.05738169550349262, victory 0.05649919814367649, palm 0.05253840547466611, wreath 0.05082142208153891, advancing 0.03642126606622154, jupiter 0.030048335900069785, winged 0.025424828723638795, extended 0.024984171280504886, caduceus 0.023408682926464842, thunderbolt 0.02037443954492004, felicitas 0.019139390694642842, cornucopiae 0.018757363095869242, antoninuspius 0.01783932735654865, feet 0.016177396404340974, domitian 0.016038791561847697, septimiusseverus 0.015267652547054455, eagle 0.014979324861764327, gallienus 0.012232645566130896, valerian 0.00991602215798045, venus 0.009261319900296276]"
4,"[emperor 0.07109937991988029, victory 0.06385290035379859, globe 0.05573777574968814, captive 0.05432417184679488, labarum 0.03984369639467067, advancing 0.038882132624450824, dragging 0.03323673398758548, cross 0.03088591959805974, valentiniani 0.0306908487647795, standard 0.023895071575307857, hercules 0.02136168606953911, valens 0.020797572306075072, club 0.02032014752432319, wreath 0.018860828585207715, field 0.01868272664026337, foot 0.01630366098908093, long 0.01472110250649863, theodosiusi 0.014146710735104127, winged 0.01262045134205413, star 0.011926230869223182]"
5,"[shield 0.05938984996306914, spear 0.05727033794472761, helmeted 0.03986869605676984, victory 0.03296967477722263, resting 0.019258976690932874, globe 0.018717608836199096, winged 0.018469164846977946, gallienus 0.017212788939039627, roma 0.016372325053449308, sceptre 0.015055195974641764, military 0.013365691557876874, constantiusii 0.012923064442012093, wreath 0.012802028033113526, mars 0.01256898176851692, soldier 0.01234132095713391, trophy 0.012037409121119629, constantinei 0.011753050886446742, advancing 0.010936652542894078, arm 0.010448681019765586, foot 0.009848173824575548]"
6,"[sceptre 0.060364246966633936, pax 0.047539406734559, scales 0.03798798162479387, branch 0.035714775157328904, cornucopiae 0.033888991040258575, aequitas 0.025702160447007353, olivebranch 0.024008310101981845, carausius 0.022954976310238715, vespasian 0.021404696375193148, hadrian 0.01981461686654317, globe 0.018732506904440942, constans 0.01870839358691975, galley 0.018417085973427023, standard 0.0158010938